In [13]:
"""TensorFlow 2.0 implementation of vanilla Autoencoder."""

__version__ = '1.0.0'
__author__ = 'Abien Fred Agarap'

import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.random.set_seed(1)
batch_size = 128
epochs = 10
learning_rate = 1e-2
intermediate_dim = 64
original_dim = 784

(training_features, _), _ = tf.keras.datasets.mnist.load_data()
training_features = training_features / np.max(training_features)
training_features = training_features.reshape(training_features.shape[0],
                                              training_features.shape[1] * training_features.shape[2])
training_features = training_features.astype('float32')

training_dataset = tf.data.Dataset.from_tensor_slices(training_features)
training_dataset = training_dataset.batch(batch_size)
training_dataset = training_dataset.shuffle(training_features.shape[0])
training_dataset = training_dataset.prefetch(batch_size * 4)


class Encoder(tf.keras.layers.Layer):
    def __init__(self, intermediate_dim):
        super(Encoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu, kernel_initializer='he_uniform')
        self.output_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.sigmoid)
        
    def call(self, input_features):
        activation = self.hidden_layer(input_features)
        return self.output_layer(activation)

class Decoder(tf.keras.layers.Layer):
    def __init__(self, intermediate_dim, original_dim):
        super(Decoder, self).__init__()
        self.hidden_layer = tf.keras.layers.Dense(units=intermediate_dim, activation=tf.nn.relu, kernel_initializer='he_uniform')
        self.output_layer = tf.keras.layers.Dense(units=original_dim, activation=tf.nn.sigmoid)
        
    def call(self, code):
        activation = self.hidden_layer(code)
        return self.output_layer(activation)

In [15]:
%load_ext tensorboard
!rm -rf ./tmp

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
%tensorboard --logdir ./tmp

Reusing TensorBoard on port 6006 (pid 28356), started 0:00:01 ago. (Use '!kill 28356' to kill it.)

In [18]:
(Check, _), _ = tf.keras.datasets.mnist.load_data()

In [19]:
Check.shape

(60000, 28, 28)

In [21]:
class Autoencoder(tf.keras.Model):
    def __init__(self, intermediate_dim, original_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(intermediate_dim=intermediate_dim)
        self.decoder = Decoder(intermediate_dim=intermediate_dim,original_dim=original_dim)
  
    def call(self, input_features):
        code = self.encoder(input_features)
        reconstructed = self.decoder(code)
        return reconstructed

autoencoder = Autoencoder(intermediate_dim=intermediate_dim, original_dim=original_dim)
opt = tf.optimizers.Adam(learning_rate=learning_rate)

def loss(model, original):
    reconstruction_error = tf.reduce_mean(tf.square(tf.subtract(model(original), original)))
    return reconstruction_error
  
def train(loss, model, opt, original):
    with tf.GradientTape() as tape:
        gradients = tape.gradient(loss(model, original), model.trainable_variables)
    gradient_variables = zip(gradients, model.trainable_variables)
    opt.apply_gradients(gradient_variables)

writer = tf.summary.create_file_writer('tmp')

with writer.as_default():
    with tf.summary.record_if(True):
        for epoch in range(epochs):
            for step, batch_features in enumerate(training_dataset):
                train(loss, autoencoder, opt, batch_features)
                loss_values = loss(autoencoder, batch_features)
                if step%100 == 0:
                    print('{}/{}: loss Values: {}'.format(epoch, step, loss_values))
                original = tf.reshape(batch_features, (batch_features.shape[0], 28, 28, 1))
                reconstructed = tf.reshape(autoencoder(tf.constant(batch_features)), (batch_features.shape[0], 28, 28, 1))
                tf.summary.scalar('loss', loss_values, step=step)
                tf.summary.image('original', original, max_outputs=10, step=step)
                tf.summary.image('reconstructed', reconstructed, max_outputs=10, step=step)
                

0/0: loss Values: 0.19928951561450958
0/100: loss Values: 0.05247677490115166
0/200: loss Values: 0.04047640413045883
0/300: loss Values: 0.03306388482451439
0/400: loss Values: 0.029457123950123787
1/0: loss Values: 0.030749019235372543
1/100: loss Values: 0.030711976811289787
1/200: loss Values: 0.024476367980241776
1/300: loss Values: 0.021552864462137222
1/400: loss Values: 0.019970903173089027
2/0: loss Values: 0.021942138671875
2/100: loss Values: 0.01911173388361931
2/200: loss Values: 0.019467180594801903
2/300: loss Values: 0.017186179757118225
2/400: loss Values: 0.01748153381049633
3/0: loss Values: 0.017791911959648132
3/100: loss Values: 0.016490096226334572
3/200: loss Values: 0.019367534667253494
3/300: loss Values: 0.016727155074477196
3/400: loss Values: 0.01811329647898674
4/0: loss Values: 0.017989767715334892
4/100: loss Values: 0.01672404073178768
4/200: loss Values: 0.017032813280820847
4/300: loss Values: 0.014861253090202808
4/400: loss Values: 0.014952255412936